In [142]:
import re
import os
import pickle
import snowflake.connector
from snowflake.connector.pandas_tools import write_pandas
import pandas as pd
import numpy as np

from transformers import AutoModelForSequenceClassification, AutoTokenizer
from sentence_transformers import SentenceTransformer, util

import statsmodels.api as sm
import statsmodels.formula.api as smf
from scipy.optimize import minimize


In [143]:
# Get SQL query file as data input 
def get_sql_file_as_text(file_path):
  
  with open(file_path, 'r') as f:
    sql_content = f.read()
  return sql_content

# Establish connection with Snowflake
conn = snowflake.connector.connect(connection_name="fundingsociety.sg.ap-southeast-1.aws.privatelink")

In [144]:
# Write SQL Snowflake Query here
sql_file = "for loglog.sql"
sql_text = get_sql_file_as_text(sql_file)
my_query=sql_text

# Get data from query above
df = conn.cursor().execute(my_query).fetch_pandas_all()

In [145]:
df.head()

,MONTH_TRANSACTED,COMPANY_ID,CARD_TYPE,INDUSTRY,NEXT_DAY,TOTAL_GTV,TOTAL_NET_REVENUE,CARDUP_FEE_RATE,COST_RATE,GTV_BUCKET
0,2024-10-01,3401,Visa,Business Services,0,55836.825000000000,221.902987460000,0.0170,0.013025866667,"01. $50-60,000"
1,2024-03-01,3401,Visa,Business Services,0,68195.375600000000,340.247276820000,0.0170,0.012010698690,"01. $60-70,000"
2,2024-04-01,3401,Visa,Business Services,0,23674.813800000000,167.994394150000,0.0170,0.009904088050,"01. $20-30,000"
3,2023-05-01,1944,Visa,"Energy, Environment and Resources",0,107961.550869810000,496.121357490000,0.0185,0.013904528193,"02. $100-110,000"
4,2023-06-01,1944,Visa,"Energy, Environment and Resources",0,102300.508310000000,537.939416960000,0.0185,0.013241612692,"02. $100-110,000"


In [146]:
df['TOTAL_GTV'] = df['TOTAL_GTV'].astype(float)
df['TOTAL_NET_REVENUE'] = df['TOTAL_NET_REVENUE'].astype(float)
df['CARDUP_FEE_RATE'] = df['CARDUP_FEE_RATE'].astype(float)

In [147]:
df_regression = df.copy()

In [148]:

# Convert to log-log scale
df_regression['log_TOTAL_GTV'] = np.log(df_regression['TOTAL_GTV'])
df_regression['log_CARDUP_FEE_RATE'] = np.log(df_regression['CARDUP_FEE_RATE'])

# Fixed Effects: Company & Month as categorical
df_regression['COMPANY_ID'] = df_regression['COMPANY_ID'].astype(str)  # Ensure categorical
df_regression['MONTH_TRANSACTED'] = df_regression['MONTH_TRANSACTED'].astype(str)

In [149]:
# Create lagged variables
df_regression['log_TOTAL_GTV_lag'] = df_regression.groupby('COMPANY_ID')['log_TOTAL_GTV'].shift(1)
df_regression['TOTAL_NET_REVENUE_lag'] = df_regression.groupby('COMPANY_ID')['TOTAL_NET_REVENUE'].shift(1)
df_regression['log_CARDUP_FEE_RATE_lag'] = df_regression.groupby('COMPANY_ID')['log_CARDUP_FEE_RATE'].shift(1)

In [150]:
df_regression.head()

,MONTH_TRANSACTED,COMPANY_ID,CARD_TYPE,INDUSTRY,NEXT_DAY,TOTAL_GTV,TOTAL_NET_REVENUE,CARDUP_FEE_RATE,COST_RATE,GTV_BUCKET,log_TOTAL_GTV,log_CARDUP_FEE_RATE,log_TOTAL_GTV_lag,TOTAL_NET_REVENUE_lag,log_CARDUP_FEE_RATE_lag
0,2024-10-01,3401,Visa,Business Services,0,55836.82500,221.902987,0.0170,0.013025866667,"01. $50-60,000",10.930189,-4.074542,NaN,NaN,NaN
1,2024-03-01,3401,Visa,Business Services,0,68195.37560,340.247277,0.0170,0.012010698690,"01. $60-70,000",11.130132,-4.074542,10.930189,221.902987,-4.074542
2,2024-04-01,3401,Visa,Business Services,0,23674.81380,167.994394,0.0170,0.009904088050,"01. $20-30,000",10.072167,-4.074542,11.130132,340.247277,-4.074542
3,2023-05-01,1944,Visa,"Energy, Environment and Resources",0,107961.55087,496.121357,0.0185,0.013904528193,"02. $100-110,000",11.589530,-3.989985,NaN,NaN,NaN
4,2023-06-01,1944,Visa,"Energy, Environment and Resources",0,102300.50831,537.939417,0.0185,0.013241612692,"02. $100-110,000",11.535670,-3.989985,11.589530,496.121357,-3.989985


In [151]:
df_regression_lag = df_regression.dropna(subset=['log_TOTAL_GTV_lag', 'log_CARDUP_FEE_RATE_lag', 'TOTAL_NET_REVENUE_lag'])

In [152]:
# Run Log-Log Regression with Lagged Variables and Interactions
model_lag = smf.ols(
    'log_TOTAL_GTV ~ log_CARDUP_FEE_RATE + log_CARDUP_FEE_RATE_lag + '
    '(log_CARDUP_FEE_RATE:NEXT_DAY) + ' 
    '(log_CARDUP_FEE_RATE:COMPANY_ID) + '
    'log_TOTAL_GTV_lag + '  
    'NEXT_DAY + C(COMPANY_ID) + C(MONTH_TRANSACTED)',
    data=df_regression_lag
).fit()

# Print summary
print(model_lag.summary())

                            OLS Regression Results                            
Dep. Variable:          log_TOTAL_GTV   R-squared:                       0.779
Model:                            OLS   Adj. R-squared:                  0.715
Method:                 Least Squares   F-statistic:                     12.22
Date:                Thu, 06 Mar 2025   Prob (F-statistic):               0.00
Time:                        10:04:10   Log-Likelihood:                -4534.1
No. Observations:                3612   AIC:                         1.068e+04
Df Residuals:                    2804   BIC:                         1.569e+04
Df Model:                         807                                         
Covariance Type:            nonrobust                                         
                                             coef    std err          t      P>|t|      [0.025      0.975]
-------------------------------------------------------------------------------------------------------

In [153]:
# Example - Extract Price Elasticity for Different Segments - Model with Lag
base_elasticity = model_lag.params['log_CARDUP_FEE_RATE']
next_day_elasticity = base_elasticity + model_lag.params.get('log_CARDUP_FEE_RATE:NEXT_DAY', 0)

print(f'Base Price Elasticity: {base_elasticity:.3f}')
print(f'Next-Day Price Elasticity: {next_day_elasticity:.3f}')

Base Price Elasticity: -3.124
Next-Day Price Elasticity: -2.703


In [154]:
import pandas as pd

# Get the coefficients and p-values from the model_lag
coefficients = model_lag.params
p_values = model_lag.pvalues

# Create a dataframe
df_coefficients = pd.DataFrame({'Variable': coefficients.index, 'Coefficient': coefficients.values, 'P-value': p_values.values})

# Save the dataframe as a CSV file
df_coefficients.to_csv('model_lag_coefficients.csv', index=False)


## Net Rev

In [155]:
# Define GTV tiers
bins = [0, 100000, 150000, 200000, float('inf')]  # Define your bins based on GTV distribution
labels = ['Tier 1', 'Tier 2', 'Tier 3', 'Tier 4']

# Use .loc to assign the new column to the DataFrame
df_regression_lag.loc[:, 'GTV_Tier'] = pd.cut(df_regression_lag['TOTAL_GTV'], bins=bins, labels=labels)

/var/folders/sp/wnd46psj0r5_ys790_zv7xgm0000gr/T/ipykernel_30271/3414894510.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_regression_lag.loc[:, 'GTV_Tier'] = pd.cut(df_regression_lag['TOTAL_GTV'], bins=bins, labels=labels)
